In [1]:
import sqlite3
import random
!pip install faker
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker for generating realistic fake names and timestamps
fake = Faker()

# Connect to SQLite (creates the file if it doesn’t exist)
conn = sqlite3.connect("library_bookings.db")
c = conn.cursor()

# Enable foreign key constraints
c.execute("PRAGMA foreign_keys = ON;")

# ------------------------------------------------------------
# 1️ Create all tables (restarting from scratch if needed)
# ------------------------------------------------------------
c.executescript("""
DROP TABLE IF EXISTS Bookings;
DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Rooms;

CREATE TABLE Student (
    student_id     INTEGER PRIMARY KEY,
    full_name      TEXT NOT NULL,
    student_level  TEXT NOT NULL CHECK(student_level IN ('UG', 'PG')),
    study_year     INTEGER NOT NULL CHECK(study_year BETWEEN 1 AND 5),
    register_date  DATE NOT NULL
);

CREATE TABLE Room (
    room_id    INTEGER PRIMARY KEY,
    room_name  TEXT UNIQUE NOT NULL,
    capacity   INTEGER NOT NULL CHECK(capacity >= 1 AND capacity <= 50),
    room_type  TEXT NOT NULL CHECK(room_type IN ('quiet','group','computer'))
);

CREATE TABLE Booking (
    student_id       INTEGER NOT NULL,
    room_id          INTEGER NOT NULL,
    booking_datetime DATETIME NOT NULL,
    duration_hours   REAL NOT NULL CHECK(duration_hours > 0 AND duration_hours <= 5),
    attendance       TEXT CHECK(attendance IN ('show','no_show') OR attendance IS NULL),
    purpose          TEXT CHECK(purpose IN ('study','meeting','exam_prep','other') OR purpose IS NULL),

    -- Composite primary key: ensures a student cannot book the same room at the same datetime
    PRIMARY KEY (student_id, room_id, booking_datetime),

    -- Foreign key constraints linking Booking to Student and Room
    FOREIGN KEY (student_id) REFERENCES Student(student_id) ON DELETE CASCADE,
    FOREIGN KEY (room_id) REFERENCES Room(room_id) ON DELETE CASCADE,

    -- Prevents double-booking of the same room at the same time
    UNIQUE (room_id, booking_datetime)
);
""")

# ------------------------------------------------------------
# 2️ Insert random students
# ------------------------------------------------------------
student_levels = ["UG", "PG"]
students = []

for i in range(1, 51):  # Generate 50 students
    name = fake.name()
    level = random.choice(student_levels)
    year = random.randint(1, 5)
    reg_date = fake.date_between(start_date='-3y', end_date='today')
    students.append((i, name, level, year, reg_date))

c.executemany("INSERT INTO Student VALUES (?, ?, ?, ?, ?)", students)
print(" Inserted 50 students")

# ------------------------------------------------------------
# 3️ Insert random rooms
# ------------------------------------------------------------
room_types = ["quiet", "group", "computer"]
rooms = []

for i in range(1, 11):  # Generate 10 rooms
    name = f"Room {i}"
    capacity = random.randint(4, 20)
    rtype = random.choice(room_types)
    rooms.append((i, name, capacity, rtype))

c.executemany("INSERT INTO Room VALUES (?, ?, ?, ?)", rooms)
print(" Inserted 10 rooms")

# ------------------------------------------------------------
# 4️ Generate and insert 1000 random bookings
# ------------------------------------------------------------
purposes = ["study", "meeting", "exam_prep", "other"]
attendance_values = ["show", "no_show", None]

bookings = set()  # used to avoid duplicate composite keys
n_rows = 1000

while len(bookings) < n_rows:
    student_id = random.randint(1, 50)
    room_id = random.randint(1, 10)

    # Generate a booking date/time within the last 6 months
    booking_time = fake.date_time_between(start_date='-180d', end_date='now')

    # Booking duration between 0.5 and 5 hours
    duration = round(random.uniform(0.5, 5.0), 1)

    # Attendance may be 'show', 'no_show', or still pending (NULL)
    attendance = random.choice(attendance_values)

    # Purpose of booking: study, meeting, exam prep, or other
    purpose = random.choice(purposes)

    # Prevent duplicate (student_id, room_id, booking_time)
    key = (student_id, room_id, booking_time)
    if key not in bookings:
        bookings.add(key)
        c.execute(
            "INSERT INTO Booking VALUES (?, ?, ?, ?, ?, ?)",
            (student_id, room_id, booking_time, duration, attendance, purpose)
        )

print(f" Inserted {len(bookings)} bookings")

# ------------------------------------------------------------
# 5️ Commit and close the connection
# ------------------------------------------------------------
conn.commit()
conn.close()

print(" Database 'library_bookings.db' successfully created and populated!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00
 Inserted 50 students
 Inserted 10 rooms
 Inserted 1000 bookings
 Database 'library_bookings.db' successfully created and populated!


/tmp/ipython-input-3238131724.py:73: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  c.executemany("INSERT INTO Student VALUES (?, ?, ?, ?, ?)", students)
/tmp/ipython-input-3238131724.py:120: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  c.execute(


In [3]:
!ls -lh /content


total 168K
-rw-r--r-- 1 root root 164K Nov  7 16:22 library_bookings.db
drwxr-xr-x 1 root root 4.0K Nov  5 14:33 sample_data
